In [31]:
import pandas as pd
import random
from time import sleep
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import numpy as np
import nltk
import os
from sklearn import datasets
import pickle
from nltk.tokenize import word_tokenize
from sklearn.decomposition import NMF, TruncatedSVD
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.neighbors import NearestNeighbors
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, roc_curve, auc, fbeta_score
from sklearn.model_selection import learning_curve, KFold, GridSearchCV, cross_val_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.dummy import DummyClassifier
from collections import Counter
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.preprocessing import normalize
import re
from unidecode import unidecode

import matplotlib.style as style
%matplotlib inline
plt.style.use('ggplot')

In [32]:
with open('data/cleaned_df.pkl', 'rb') as picklefile:
    df = pickle.load(picklefile)

In [33]:
df.head()

,song_owner,song_title,song_element,verse,artist
0,Childish Gambino,This Is America,[Bridge: Childish Gambino & Young Thug],"We just wanna party, Party just for you, We ju...",Childish Gambino
1,Childish Gambino,This Is America,[Chorus: Childish Gambino],"This is America, Don't catch you slippin' now,...",Childish Gambino
2,Childish Gambino,This Is America,"[Verse 1: Childish Gambino, Blocboy JB, Slim J...","This is America (skrrt, skrrt, woo), Don't cat...",Childish Gambino
3,Childish Gambino,This Is America,[Refrain: Choir & Childish Gambino],"Ooh-ooh-ooh-ooh-ooh, tell somebody, You go tel...",Childish Gambino
4,Childish Gambino,This Is America,"[Chorus: Childish Gambino, Slim Jxmmi & Young ...","This is America (woo, ayy), Don't catch you sl...",Childish Gambino


In [34]:
model_df = df.copy()

model_df = model_df[model_df['artist'] == 'Kanye West']

In [35]:
model_df['verse_words'] = model_df['verse'].copy()
model_df['verse_words'] = model_df['verse_words'].str.lower()
model_df['verse_words'] = model_df['verse_words'].apply(lambda x: x.replace(',',''))
model_df['verse_words'] = model_df['verse_words'].apply(lambda x: x.split(' '))

In [36]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words
913,Chance The Rapper,All We Got,[Outro: Kanye West & Chicago Children's Choir],"We know, we know we got it, We know, we know w...",Kanye West,"[we, know, we, know, we, got, it, we, know, we..."
1281,Chance The Rapper,All We Got,[Outro: Kanye West & Chicago Children's Choir],"We know, we know we got it, We know, we know w...",Kanye West,"[we, know, we, know, we, got, it, we, know, we..."
1746,Drake,Pop Style (Single Version),[Verse 2: Kanye West],"They like, ""Pablo—"", ""Why are all the windows ...",Kanye West,"[they, like, ""pablo—"", ""why, are, all, the, wi..."
1931,Drake,Glow,[Chorus: Kanye West],"Watch out for me, I'm bound to glow, You bette...",Kanye West,"[watch, out, for, me, i'm, bound, to, glow, yo..."
1932,Drake,Glow,[Verse 1: Kanye West],"Your boy all glowed up, Every time we touch do...",Kanye West,"[your, boy, all, glowed, up, every, time, we, ..."


In [37]:
from nltk.tokenize import word_tokenize
model_df['tokenize_verse'] = model_df['verse'].copy()
model_df['tokenize_verse'] = model_df['tokenize_verse'].str.lower()
model_df['tokenize_verse'] = model_df['tokenize_verse'].apply(lambda x: word_tokenize(x))

In [38]:
model_df['verse'] = model_df['verse'].str.lower()
model_df['verse'] = model_df['verse'].apply(lambda x: x.replace("'",''))
model_df['verse'] = model_df['verse'].apply(lambda x: "".join([c if c.isalnum() else " " for c in x ]))
model_df['verse'] = model_df['verse'].apply(lambda x: x.replace('  ',' '))

In [39]:
stopwords = ["yeah", "uh", "huh", "aw", "oh", "la", "ayy",
            "na", "ha", "ay", "skrrt",
            "em", "bla", "woo", "wu", "ya", "wala", "woop", "wrarrr", "ooh",
            "ba", "duh", "du", "1", "2", "3", "4" ,"5" ,"6", "7", "8", "9", "0"]

for index, row in model_df.iterrows():
    querywords = row['verse'].split()

    resultwords  = [word for word in querywords if word.lower() not in stopwords]
    row['verse'] = ' '.join(resultwords)

model_df.verse[:5]

913     we know we know we got it we know we know we g...
1281    we know we know we got it we know we know we g...
1746    they like pablo why are all the windows tinted...
1931    watch out for me im bound to glow you better w...
1932    your boy all glowed up every time we touch dow...
Name: verse, dtype: object

In [40]:
verses = list(model_df.verse.values)

In [41]:
verses[:10]

['we know we know we got it we know we know we got it we know we know we got it we know we know we got it music we know we know we got it we know we know we got it we know we know we got it we know we know we got it music music is all we got',
 'we know we know we got it we know we know we got it we know we know we got it we know we know we got it music we know we know we got it we know we know we got it we know we know we got it we know we know we got it music music is all we got',
 'they like pablo why are all the windows tinted on your tahoe why do you know every single bitch that i know why cant you just shut your mouth and take the high road fuck if i know thats that chicago south south side thats the motto cop a crib and spend ten million on remodel take the devils out my life and preach the gospel cause i know we went way way past the line of scrimmage throne is back up in it in the field like emmitt yall get so offended i be blacking out i aint backing out jay about his busines

In [42]:
all_verses_all_rappers = ','.join(verses)

all_verses_all_rappers[:100]

'we know we know we got it we know we know we got it we know we know we got it we know we know we got'

In [43]:
file = all_verses_all_rappers

In [44]:
# Read in text and change unicode characters to ASCII
import unidecode
import string
import random
import re

all_characters = string.printable
n_characters = len(all_characters)

file_len = len(file)
print(f'file_len = {file_len}')

file_len = 692640


In [45]:
chunk_len = 400

def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

and like a picture in a frame put it against the wall said she had never been with a man before but she not a virgin she a lady lover i didnt know yall take each other out and date each other but thats a lady lover i aint even know that was baby mother,explain the outro man dont nobody never get to the outro i just take a whole record and just im not gonna talk about the producer shit right now i j


In [46]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, hidden):
        output = self.encoder(input.view(1, -1))
        output, hidden = self.gru(output.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        
        return output, hidden
    
    def init_hidden(self):
        return Variable(torch.randn(self.n_layers, 1, self.hidden_size))

In [47]:
def char2tensor(string):
    tensor = torch.zeros(len(string)).long()
    for char in range(len(string)):
        if string[char] in all_characters:
            tensor[char] = all_characters.index(string[char])
        else:
            tensor[char] = 94 #predict space if character unknown
        
    return Variable(tensor)

# Let's see it in action.
print(char2tensor('Metis0123abczABC'))

tensor([48, 14, 29, 18, 28,  0,  1,  2,  3, 10, 11, 12, 35, 36, 37, 38])


In [48]:
def random_training_set():
    chunk = random_chunk()
    inp = char2tensor(chunk[:-1])
    target = char2tensor(chunk[1:])
    return inp, target

In [49]:
def evaluate(model, prime_str='A', predict_len=100, temperature=0.8):
    
    hidden = model.init_hidden()
    prime_input = char2tensor(prime_str)
    predicted = prime_str
    
    # use priming string to build up hidden state
    
    for p in range(len(prime_str) - 1):
        _, hidden = model(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = model(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char2tensor(predicted_char)
        
    return predicted

In [50]:
# helper function
import time, math

def time_since(since):
    s = time.time() - since
    m = math.floor(s/60)
    s -= m*60
    return '%dm %ds' % (m, s)

In [51]:
# The actual training part
def train(inp, target):
    hidden = model.init_hidden()
    model.zero_grad()
    loss = 0

    for char in range(chunk_len):
        output, hidden = model(inp[char], hidden)
        loss += criterion(output, target[char].unsqueeze(0))

    loss.backward()
    model_optimizer.step()

    return loss.data.item() / chunk_len

In [52]:
# parameters
# n_epochs = 1000
# print_every = 100
# plot_every = 10
# hidden_size = 256
# n_layers = 2
# learning_rate = 0.001

# # model declaration
# model = RNN(n_characters, hidden_size, n_characters, n_layers)
# model_optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# criterion = nn.CrossEntropyLoss()

# start = time.time()
# all_losses = []
# loss_avg = 0

# for epoch in range(1, n_epochs + 1):
#     loss = train(*random_training_set())       
#     loss_avg += loss

#     if epoch % print_every == 0:
#         print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
#         print(evaluate(model, 'A ', 100), '\n')

#     if epoch % plot_every == 0:
#         all_losses.append(loss_avg / plot_every)
#         loss_avg = 0

In [53]:
# import matplotlib.pyplot as plt
# import matplotlib.ticker as ticker
# %matplotlib inline

# plt.figure()
# plt.plot(all_losses)

In [54]:
# with open('models/kanye_rnn_generator.pkl', 'wb') as picklefile:
#     pickle.dump(model, picklefile)

In [55]:
# Evaluate model from this notebook
print(evaluate(model, 'To be or not to be: ', predict_len=200, temperature=0.5))

To be or not to be: all the make i get a to me like the shit make min but the clothes and the game i want the cant get off the come is the mall tell you will i can the rain to parrother i could the talked the starter the


In [56]:
# Evaluate model from this notebook
print(evaluate(model, 'I miss you girl come back ', predict_len=200, temperature=0.5))

I miss you girl come back and for the mother the craiches right the alright the game now i her the long the recame with is the chass and the something they can it off the life the life that you came the classic and it off the 


In [57]:
# Evaluate model from this notebook
print(evaluate(model, "Started from the bottom", predict_len=200, temperature=0.4))

Started from the bottom the start the back the never the champany a cray nigga that i m get that you so he down and i all the comple in the vomie i dont tell you could so i aint to get the prother the have that the could an


In [58]:
# Evaluate model from this notebook
print(evaluate(model, "I'm the realest rapper alive", predict_len=200, temperature=0.4))

I'm the realest rapper alive a champany that the say like that what i can me all the more they say the cant that the look the because i cant this the cant and the runnin the like the do it a call you do the chi and this it the c


In [59]:
# Evaluate model from this notebook
print(evaluate(model, "Smoke a lot of weed", predict_len=200, temperature=0.4))

Smoke a lot of weed to the probe this that why i aint off the like that the real that its that that the the chool to they dont get a but i aint the mother that shit off these with the dont get that i want the come where


In [60]:
# Evaluate model from this notebook
print(evaluate(model, "Baby come", predict_len=200, temperature=0.4))

Baby come into the way nigga get the corner that i start that i get that im the say i got that i m cant get the compan i got the storth and that you alright i want the real back i got the pay i get a compan th


In [61]:
prompts = ['I miss you girl come back ', 'To be or not to be: ', "Started from the bottom", "I'm the realest rapper alive",
           "Smoke a lot of weed", "Baby come", 'Date a scientist', 'Butter my biscuit', 'Scramble my ', 'I sleep ',
           'Green eggs and ', 'Rollin in my', 'Drop top', 'Math is fun', 'Sentences', 'Coffee in the', 'Table that',
           'Homies', 'From the streets', 'Maserati', 'Air force one', 'President', 'Trump']

kanye_rnn_verses = []

for i in range(len(prompts)):
    kanye_rnn_verses.append(evaluate(model, prompts[i], predict_len=200, temperature=0.5))
    
    
kanye_rnn_verses

['I miss you girl come back that i to the wrock the reat to be me that to the the clubs right and that she want the light if the way in i dont let thats the alright the roc this so her the corse on the pother to be i cant when i',
 'To be or not to be: and to the class like off make it to to the rom something we cause off you aint get to make the crazy that im off you like that the light what it off that the rick it the comp i parer the chance on i ',
 'Started from the bottom the round that you got why it to me nigga and it off this just get that i was the recite i got the prain est but that the like what the she comin a for that it they back shit the star that the craid ',
 "I'm the realest rapper alive the be im the look a clife to be all the cont the way i pass you know you and what you ever day not hill the way we want what that the cont p something that i can this the way that the time minate go",
 'Smoke a lot of weed the be in the doubted that the say i dont and it the rings i 

In [62]:
columns = ['generated_verses']

kanye_generated_df = pd.DataFrame(kanye_rnn_verses, columns=columns)

In [65]:
kanye_generated_df['artist'] = 'Kanye West'

kanye_generated_df

,generated_verses,artist
0,I miss you girl come back that i to the wrock ...,Kanye West
1,To be or not to be: and to the class like off ...,Kanye West
2,Started from the bottom the round that you got...,Kanye West
3,I'm the realest rapper alive the be im the loo...,Kanye West
4,Smoke a lot of weed the be in the doubted that...,Kanye West
5,Baby come i got that that how you could me off...,Kanye West
6,Date a scientistant im go do i m screation we ...,Kanye West
7,Butter my biscuit me they want my stop i m day...,Kanye West
8,Scramble my cant off that that the bent i was ...,Kanye West
9,I sleep the way i know now the lost the please...,Kanye West


In [66]:
with open('data/kanye_generated_raps.pkl', 'wb') as picklefile:
    pickle.dump(kanye_generated_df, picklefile)